In [ ]:
#SageMath version 9.2

import numpy as np
from sympy import *

G = str(input("Enter group name as 'Capital letter + number', e.g. A2,A4,F4,G2,E6,..."))

rankG = int((lie.Lie_rank(G))) #rank of the group

def fullchar(v, g):
    '''This function returns the full character, with arguments weight and group.'''
    return lie.W_orbit(lie.dom_char(v,g),g)

def coeffs(p):
    '''This function takes a numpy polynomial argument and returns its coefficients as a list of integers'''
    length = int(lie.length(p))
    coeffs =[int(lie.coef(p,i+1)) for i in range(length)]
    return coeffs

def exponents(p):
    '''This function takes a numpy polynomial argument and returns its exponents as a list of integers'''
    length = int(lie.length(p))
    return [list(map(int,lie.expon(p,i+1))) for i in range(length)]


#List which will hold the fundamental weights.
fund_weights_list = [list(map(int,np.insert(np.zeros(rankG-1),i,1))) for i in range(rankG)] 

fund_polys_coeff_list = []
fund_polys_exponent_list = []

for k,wght in enumerate(fund_weights_list):
    p = fullchar(fund_weights_list[k],G)
    fund_polys_coeff_list.append(coeffs(p))
    fund_polys_exponent_list.append(exponents(p))

class LengthError(Exception): #exception for later when input length is not the right size
    pass

go_again = 'y'

while go_again != 'n':
    while True:
        try:
            Rval = [int(item) for item in input("Enter the value of H at each root (ordering convention \
as in Humphreys') each seperated by a space: ").split()]
            if len(Rval) != rankG:
               raise LengthError
            break
        except LengthError:
            print("You have input the wrong number of integers. Please insert "+str(rankG)+" integers, \
each seperated by space.")
        except ValueError:
            print("One or more entries were not of type int. Please try again.")
    while True:
        #ask the user for the characteristic of prime field they wish to calculate K over.
        try:
            char = int(input("Enter characteristic of prime field you want K^{{0,0}} calculated over."))
            break
        except ValueError:
            print("Please insert a prime or 0")

    #We now have a list of integers telling us how H acts on roots, 
    #however we need to know how H acts on fundamental weights. 
    #To make this conversion we use the cartan matrix, or rather it's inverse
    #(weight values)^T = C^{-1}(root values)^T

    #Cartan matrix of G above. This code writes Cartan as integer array
    #so np.linalg.inv can invert it
    C = [list(map(int,lie.Cartan(G)[i])) for i in range(1,rankG+1)]
    Cinv = np.linalg.inv(C)
    
    #action of H on the weights i.e. C^{-1}(H on simple roots)
    Wval = list(map(float,Cinv.dot(np.array(Rval))))
    
    #We now create a "master list" of the restricted character polynomials. 
    #We record each polynomial as a list of coefficients and a list of exponents
    res_poly_master_list = []
    
    for k,wght in enumerate(fund_weights_list):
        p = fullchar(wght,G)
                     
        restriction_exponents = [int(round(sum(Wval[j]*fund_polys_exponent_list[k][i][j] for j in range(rankG))))
                                 for i in range(int(lie.length(p)))]
        
        #Now we have two lists of interest:
        #fund_polys_coeff_list[k] which gives a list of coeffs from the kth character polynomial
        #restriction_exponents, a list of exponents from restricted character polynomial to sl2
        #This may look something like [1,3,2,3] and [2,-2,2,0] ie the poly 1x^2 + 3x^{-2}+2x^2+3.
        #observe that x^2 appears twice and the exponents are not in particular order in the latter list
        #now we adjust these lists to get e.g. 3x^2 + 3 + 3x^{-2} i.e [3,3,3] and [-2,0,-2]

        M,n = max(restriction_exponents),min(restriction_exponents)
        coeffs_ordered = []
        exponents_ordered = []
        for i in range(M,n-1,-1): #run through each exponent from max to min decreasing by 1 each loop
            num_appearances = 0
            for j in range(len(restriction_exponents)):
                if restriction_exponents[j] == i:
                    num_appearances += fund_polys_coeff_list[k][j]
            coeffs_ordered.append(num_appearances)
            exponents_ordered.append(i)
        res_poly_master_list.append([coeffs_ordered,exponents_ordered])
    
    x = symbols('x')
    init_printing(use_unicode=False, wrap_line=False)

    sl2_irrep_polys = [1,x] 
    #e.g. where , P_0 = 1, P_1 = x and use 
    #[P_1][P_n] = [P_{n-1}]+[P_{n+1}] to calculate all them (or as many as we need).

    max_irrep = 0 #the maximum P_i that we will need (aka the max exponent of the restricted char polys.)
    for i in range(rankG):
        if res_poly_master_list[i][1][0] >= max_irrep:
            max_irrep = res_poly_master_list[i][1][0]

    for i in range(2,max_irrep+1):
        X = sl2_irrep_polys[1]*sl2_irrep_polys[i-1] - sl2_irrep_polys[i-2]
        sl2_irrep_polys.append(expand(X))
   
    #sl2_irrep_polys is a list of polynomials P_0, P_1. 
    #Indexing is nice as sl2_irrep_polys[k] = P_k, corresponding to repn with highest weight k

    char_polys_wrt_x_coeffs = [list(map(int,np.zeros(max_irrep+1))) for i in range(rankG)]
    #this will be a list of lists of integers with index i of list k containing
    #the number of each P_i appearing in the kth fundamental representation.
        
    #The next for loop decomposes our restricted characters into sum of the P_i 
    #i.e. decompose into sum of sl2 irreps
    #Warning: This for loop wipes res_poly_master_list[i][0][j] for all i,j.    
    for i in range(rankG):
        for j in range(res_poly_master_list[i][1][0]+1):
            if res_poly_master_list[i][0][j] != 0:
                subtract = res_poly_master_list[i][0][j]
                char_polys_wrt_x_coeffs[i][res_poly_master_list[i][1][j]] += res_poly_master_list[i][0][j]
                for k in range(res_poly_master_list[i][1][j]+1):
                    res_poly_master_list[i][0][j+2*k] -= subtract
    
    char_polys_wrt_x = []
    #construct polynomials using coeffs just computed along with the sl2 irrep polynomials.
    for i in range(rankG):
        f = 0
        for j in range(max_irrep+1):
            f += char_polys_wrt_x_coeffs[i][j]*sl2_irrep_polys[j]
        char_polys_wrt_x.append(f)
    
    Y = char_polys_wrt_x[0] - int(lie.dim(fund_weights_list[0],G))
        
    for i in range(1,rankG):                 
        if char ==0:
            Y = gcd(char_polys_wrt_x[i] - int(lie.dim(fund_weights_list[i],G)),Y)
        elif char >0:
            Y = gcd(char_polys_wrt_x[i] - int(lie.dim(fund_weights_list[i],G)),Y,domain=FiniteField(char)) 

    ##########
    ##Result##
    ##########
    print("")
    print(f"K^{{0,0}}(G/SL2) \\otimes F when H acts as {Rval} on the simple roots is...")
    print("")
    print(f"R(H) \\otimes F/({Y})")
    print("")
    print("where R(H) = Z[x] if H=SL2 and R(H) = Z[x^2] if H = PSL2")
    print("")
    print(f"Would you like to input another weighted Dynkin diagram for {G}? ('n' for no, anything else for yes)")
    go_again = str(input())